<a href="https://colab.research.google.com/github/Turbocaspal/AutoGPT/blob/master/Transkript_o_Vits_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transkript-o-Vits AssemblyAI
Dieses Notebook ermöglicht die automatische Transkription und die Erstellung eines Meeting-Protokolls aus Audiodateien.

## Installation der notwendigen Bibliotheken
Im ersten Schritt werden alle notwendigen Bibliotheken wie AssemblyAI und OpenAI installiert.

In [ ]:
!pip install -U assemblyai --quiet
!pip install -U openai --quiet
!pip install markdown --quiet
!pip install tiktoken --quiet
!pip install python-docx --quiet
!pip install python-pptx --quiet
!pip install exceptions

## API-Keys einrichten
Stelle sicher, dass die API-Keys in den Colab-Secrets hinterlegt sind:
- `assemblyai`: AssemblyAI-API-Key
- `OPENAI_API_KEY`: OpenAI-API-Key

In [ ]:
# @title
from google.colab import userdata
import os
import assemblyai as aai

# AssemblyAI API-Key aus Secrets laden
aai_key = userdata.get('assemblyai')
if not aai_key:
    raise ValueError("AssemblyAI-API-Key nicht gefunden. Bitte in den Secrets unter 'assemblyai' hinterlegen.")
aai.settings.api_key = aai_key

# OpenAI API-Key aus Secrets laden
openai_key = userdata.get('OPENAI_API_KEY')
if not openai_key:
    raise ValueError("OpenAI-API-Key nicht gefunden. Bitte in den Secrets unter 'OPENAI_API_KEY' hinterlegen.")

# Ionos API-Key aus Secrets laden
openai_key = userdata.get('IONOS_API_TOKEN')
if not openai_key:
    raise ValueError("OpenAI-API-Key nicht gefunden. Bitte in den Secrets unter 'IONOS_API_TOKEN' hinterlegen.")

## Datei auswählen oder hochladen
Wähle, ob eine Datei hochgeladen werden soll, oder ob eine vorhandene Datei aus dem Verzeichnis `/content/` verwendet wird.

In [ ]:
# @title
from google.colab import files

choice = input("Möchtest du eine Datei hochladen (1) oder eine vorhandene Datei aus /content/ verwenden (2)? (Standard: 1): ") or "1"

if choice == "1":
    print("Bitte lade eine MP3- oder WAV-Datei hoch:")
    uploaded = files.upload()
    audio_file = next(iter(uploaded))
elif choice == "2":
    print("Durchsuche /content/ nach MP3- oder WAV-Dateien...")
    audio_file = None
    for file in os.listdir("/content"):
        if file.endswith(".mp3") or file.endswith(".wav") or file.endswith(".m4a") or file.endswith(".mp4"):
            audio_file = file
            break
    if not audio_file:
        raise FileNotFoundError("Keine MP3- oder WAV-Datei im Ordner /content gefunden.")
    print(f"Verwendete Datei: {audio_file}")
else:
    raise ValueError("Ungültige Auswahl. Bitte wähle entweder 1 oder 2.")

## Transkription mit AssemblyAI
Die ausgewählte Datei wird mit AssemblyAI transkribiert. Die Ergebnisse werden in einer Markdown-Datei gespeichert.

In [ ]:
# @title
# Optionen für die verbesserte Sprecherdiarisierung und Sentimentanalyse
use_sentiment = input("Soll eine Sentimentanalyse durchgeführt werden? (ja/nein, Standard: nein): ") or "nein"
use_sentiment = use_sentiment.lower() == "ja"

# Anzahl Sprecher muss immer noch abgefragt werden
speakers_expected = input("Anzahl Sprecher:")

language_code = "de"  # Sprache auf Deutsch setzen

if language_code == "de" and use_sentiment:
    print("Sentimentanalyse ist für Deutsch nicht direkt verfügbar. Sentimentanalyse wird deaktiviert.")
    use_sentiment = False

config = aai.TranscriptionConfig(
    speech_model=aai.SpeechModel.best,
    speaker_labels=True,
    speakers_expected = speakers_expected,
    language_code=language_code,
    sentiment_analysis=use_sentiment
    )

print("Starte Transkription...")
transcriber = aai.Transcriber(config=config)
transcript = transcriber.transcribe(audio_file)


# Speichern des Transkripts
if transcript.status == aai.TranscriptStatus.error:
    print(f"Fehler bei der Transkription: {transcript.error}")
else:
    transcript_file = audio_file.replace(".mp3", "_transkript.md").replace(".wav", "_transkript.md").replace(".w4a", "_transkript.md").replace(".mp4", "_transkript.md")
    print(f"Speichere Transkription als {transcript_file}...")
    with open(transcript_file, "w", encoding="utf-8") as f:
        f.write("# Transkriptionsergebnis mit Sprecherlabels\n\n")
        for utterance in transcript.utterances:
             sentiment_text = f" (Sentiment: {utterance.sentiment})" if use_sentiment else ""
             f.write(f"[{utterance.start/1000:.2f}-{utterance.end/1000:.2f}] Sprecher {utterance.speaker}: {utterance.text}{sentiment_text}\n")
    print(f"Transkription abgeschlossen! Datei gespeichert als {transcript_file}.")

## Erstellung eines Meeting-Protokolls mit LLM
Das transkribierte Ergebnis wird mithilfe der OpenAI API analysiert und in ein strukturiertes Protokoll umgewandelt.

In [ ]:
from IPython import get_ipython
from IPython.display import display
from openai import OpenAI
from google.colab import files
import glob
import os
import markdown
import random
import string
from datetime import datetime
import re
from google.colab import userdata
import tiktoken
import docx
import subprocess
import tempfile
from io import StringIO
import requests

# === API-Auswahl und -Konfiguration ===
while True:
    api_choice = input("Wähle die API:\n1: OpenAI\n2: IONOS\n")
    if api_choice in ("1", "2"):
        break
    else:
        print("Ungültige Eingabe. Bitte 1 oder 2 eingeben.")

if api_choice == "1":
    openai_key = userdata.get("OPENAI_API_KEY")
    if not openai_key:
        raise ValueError("OPENAI_API_KEY nicht in den geheimen Nutzerdaten gefunden.")
    client = OpenAI(api_key=openai_key)
    base_url = "https://api.openai.com/v1"
else:
    ionos_token = userdata.get("IONOS_API_TOKEN")
    if not ionos_token:
        raise ValueError("IONOS_API_TOKEN nicht in den geheimen Nutzerdaten gefunden.")
    client = OpenAI(api_key=ionos_token, base_url="https://openai.inference.de-txl.ionos.com/v1")

# === Verfügbare IONOS-Modelle abrufen und filtern (nur Textgenerierung) ===
available_ionos_models = {}
if api_choice == "2":
    endpoint = "https://openai.inference.de-txl.ionos.com/v1/models"
    headers = {
        "Authorization": f"Bearer {ionos_token}",
        "Content-Type": "application/json"
    }
    try:
        ionos_response = requests.get(endpoint, headers=headers).json()
        for model in ionos_response.get("data", []):
            model_id = model.get("id")
            # Modelle ausschließen, die Bilder generieren oder nur Embeddings liefern
            if "stable-diffusion" in model_id.lower() or "sentence-transformers" in model_id.lower():
                continue
            max_tokens = model.get("max_tokens")
            if not max_tokens:
                if "70B" in model_id:
                    max_tokens = 8192
                elif "405B" in model_id:
                    max_tokens = 16384
                elif "8B" in model_id:
                    max_tokens = 4096
                else:
                    max_tokens = 4096
            description = model.get("description", model_id)
            available_ionos_models[model_id] = {"name": model_id, "desc": description, "max_tokens": max_tokens}
    except Exception as e:
        print("Fehler beim Abruf der IONOS-Modelle, verwende Standardwerte.")
        available_ionos_models = {
            "meta-llama/CodeLlama-13b-Instruct-hf": {"name": "meta-llama/CodeLlama-13b-Instruct-hf", "desc": "CodeLlama 13B Instruct", "max_tokens": 4096},
            "mistralai/Mistral-7B-Instruct-v0.3": {"name": "mistralai/Mistral-7B-Instruct-v0.3", "desc": "Mistral 7B Instruct", "max_tokens": 4096},
            "meta-llama/Meta-Llama-3.1-8B-Instruct": {"name": "meta-llama/Meta-Llama-3.1-8B-Instruct", "desc": "Meta-Llama 3.1 8B Instruct", "max_tokens": 4096},
            "meta-llama/Meta-Llama-3.1-70B-Instruct": {"name": "meta-llama/Meta-Llama-3.1-70B-Instruct", "desc": "Meta-Llama 3.1 70B Instruct", "max_tokens": 8192},
            "mistralai/Mixtral-8x7B-Instruct-v0.1": {"name": "mistralai/Mixtral-8x7B-Instruct-v0.1", "desc": "Mixtral 8x7B Instruct", "max_tokens": 4096},
            "BAAI/bge-m3": {"name": "BAAI/bge-m3", "desc": "BAAI bge-m3", "max_tokens": 4096},
            "BAAI/bge-large-en-v1.5": {"name": "BAAI/bge-large-en-v1.5", "desc": "BAAI bge-large-en-v1.5", "max_tokens": 4096},
            "openGPT-X/Teuken-7B-instruct-commercial": {"name": "openGPT-X/Teuken-7B-instruct-commercial", "desc": "openGPT-X Teuken 7B instruct commercial", "max_tokens": 4096},
            "meta-llama/Meta-Llama-3.1-405B-Instruct-FP8": {"name": "meta-llama/Meta-Llama-3.1-405B-Instruct-FP8", "desc": "Meta-Llama 3.1 405B Instruct FP8", "max_tokens": 16384},
            "meta-llama/Llama-3.3-70B-Instruct": {"name": "meta-llama/Llama-3.3-70B-Instruct", "desc": "Llama 3.3 70B Instruct", "max_tokens": 8192}
        }
    # Kurze, leserliche Darstellung
    short_taglines = {
        "meta-llama/CodeLlama-13b-Instruct-hf": "CodeLlama-13b-Instruct-hf | Max Tokens: 4096 | Ideal für Code & Instruct",
        "mistralai/Mistral-7B-Instruct-v0.3": "Mistral-7B-Instruct-v0.3 | Max Tokens: 4096 | Schnell & ausgewogen",
        "meta-llama/Meta-Llama-3.1-8B-Instruct": "Meta-Llama-3.1-8B-Instruct | Max Tokens: 4096 | Kompakt & präzise",
        "meta-llama/Meta-Llama-3.1-70B-Instruct": "Meta-Llama-3.1-70B-Instruct | Max Tokens: 8192 | Für lange Texte",
        "mistralai/Mixtral-8x7B-Instruct-v0.1": "Mixtral-8x7B-Instruct-v0.1 | Max Tokens: 4096 | Multilingual",
        "BAAI/bge-m3": "BAAI/bge-m3 | Max Tokens: 4096 | Effizient & wirtschaftlich",
        "BAAI/bge-large-en-v1.5": "BAAI/bge-large-en-v1.5 | Max Tokens: 4096 | Stark in Englisch",
        "openGPT-X/Teuken-7B-instruct-commercial": "openGPT-X/Teuken-7B-instruct-commercial | Max Tokens: 4096 | Kommerziell",
        "meta-llama/Meta-Llama-3.1-405B-Instruct-FP8": "Meta-Llama-3.1-405B-Instruct-FP8 | Max Tokens: 16384 | Großer Kontext",
        "meta-llama/Llama-3.3-70B-Instruct": "Llama-3.3-70B-Instruct | Max Tokens: 8192 | Modern & dialogstark"
    }
    filtered_models = {}
    count = 1
    for model_id, info in available_ionos_models.items():
        if model_id in short_taglines:
            filtered_models[str(count)] = {"name": model_id, "short": short_taglines[model_id], "max_tokens": info["max_tokens"]}
            count += 1

# === Transkriptdatei suchen und laden ===
transcript_files = []
for ext in ["srt", "txt", "docx", "pdf", "md"]:
    transcript_files.extend(glob.glob(f"/content/*_transkript.{ext}"))
if not transcript_files:
    raise FileNotFoundError("Keine Transkriptdatei gefunden.")
transcript_file = transcript_files[0]
print(f"Verwendete Transkriptdatei: {transcript_file}")

def read_transcript_content(transcript_file):
    if transcript_file.endswith((".txt", ".md", ".srt")):
        with open(transcript_file, "r", encoding="utf-8") as file:
            return file.read()
    elif transcript_file.endswith(".docx"):
        doc = docx.Document(transcript_file)
        return "\n".join([paragraph.text for paragraph in doc.paragraphs])
    elif transcript_file.endswith(".pdf"):
        try:
            temp_file = tempfile.NamedTemporaryFile(suffix=".txt", delete=False)
            subprocess.run(["pdftotext", transcript_file, temp_file.name], check=True)
            with open(temp_file.name, "r", encoding="utf-8") as file:
                content = file.read()
            os.unlink(temp_file.name)
            return content
        except Exception as e:
            raise Exception(f"Fehler beim Lesen der PDF-Datei: {e}")

transcript_content = read_transcript_content(transcript_file)

# === CSS-Vorlage für HTML-Ausgabe ===
HTML_STYLES = """
<style>
body { font-family: Arial, sans-serif; line-height: 1.6; padding: 20px; }
h1, h2, h3, h4, h5, h6 { color: #333; }
code { background-color: #f4f4f4; padding: 2px 4px; }
</style>
"""

# === Modusauswahl ===
while True:
    try:
        mode = int(input("Wähle den Modus:\n1: Standard-Zusammenfassung\n2: Individuelle Aufgabe\n3: Ultimatives Schema\n4: Besprechungsprotokoll\n5: Auf eigene Weise\n"))
        if mode in (1, 2, 3, 4, 5):
            break
        else:
            print("Ungültige Eingabe. Bitte 1, 2, 3, 4 oder 5 eingeben.")
    except ValueError:
        print("Ungültige Eingabe. Bitte eine Zahl eingeben.")

# === Modellauswahl ===
if api_choice == "1":
    openai_models = {
        "1": "gpt-4o",
        "2": "o1",
        "3": "o3-mini"
    }
    model_prompt = "Wähle ein OpenAI-Modell:\n"
    for key in openai_models:
        model_prompt += f"{key}: {openai_models[key]}\n"
    model_prompt += "4: Benutzerdefinierte Eingabe\n"
else:
    model_prompt = "Verfügbare IONOS-Modelle (nur Textgenerierung):\n"
    for num, model_info in filtered_models.items():
        model_prompt += f"{num}: {model_info['short']}\n"
    model_prompt += "8: Benutzerdefinierte Eingabe\n"

while True:
    model_choice = input(model_prompt)
    if api_choice == "1" and model_choice in openai_models:
        break
    elif api_choice == "2" and (model_choice in filtered_models or model_choice in ("8",)):
        break
    print("Ungültige Eingabe. Bitte eine gültige Option wählen.")

if api_choice == "1":
    if model_choice == "4":
        selected_model = input("Gib den benutzerdefinierten Modellnamen ein: ")
    else:
        selected_model = openai_models[model_choice]
else:
    if model_choice == "8":
        selected_model = input("Gib den benutzerdefinierten Modellnamen ein: ")
    else:
        selected_model = filtered_models[model_choice]["name"]

# === Hilfsfunktionen für Schlüsselwörter und Dateinamen ===
def extract_keyword(text):
    words = re.findall(r'\b\w+\b', text.lower())
    word_freq = {}
    for word in words:
        if len(word) > 3 and word.isalpha():
            word_freq[word] = word_freq.get(word, 0) + 1
    if word_freq:
        return max(word_freq, key=word_freq.get)
    return "Thema"

def generate_creative_keyword(keyword, model_name):
    adjectives = [
        "strahlend", "nebulös", "tiefgründig", "kristallklar", "abenteuerlich",
        "geheimnisvoll", "dynamisch", "phantastisch", "wunderbar", "einzigartig",
        "lebendig", "verspielt", "episch", "harmonisch", "unendlich"
    ]
    random_adjective = random.choice(adjectives)
    return f"{random_adjective}-{model_name.replace('/', '-')}-{keyword}".replace(" ", "-").lower()

def merge_protocol_parts(protocol_parts, model, selected_model):
    separator = "\n\n"
    merge_prompt = f"""Fasse die folgenden Protokollteile zu einem kohärenten, zusammenhängenden Besprechungsprotokoll zusammen.
Achte darauf, dass alle wesentlichen Informationen, Entscheidungen, offenen Fragen, Lessons Learned, Metaphern und
Agenda-Punkte (für den nächsten Termin) klar strukturiert und übersichtlich dargestellt werden.

Protokollteile:
{separator.join(protocol_parts)}

Erstelle daraus ein einheitliches, gut strukturiertes Protokoll:"""
    merge_messages = [{"role": "user", "content": merge_prompt}]
    response = model.chat.completions.create(
        model=selected_model,
        messages=merge_messages
    )
    return response.choices[0].message.content.strip()

def split_transcript(transcript, max_tokens, model_name):
    try:
        encoding = tiktoken.encoding_for_model(model_name)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    tokens = encoding.encode(transcript)
    if len(tokens) <= max_tokens:
        return [transcript]
    parts = []
    current_part = ""
    current_tokens = 0
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', transcript)
    for sentence in sentences:
        sentence_tokens = len(encoding.encode(sentence))
        if current_tokens + sentence_tokens <= max_tokens:
            current_part += sentence + " "
            current_tokens += sentence_tokens
        else:
            parts.append(current_part.strip())
            current_part = sentence + " "
            current_tokens = sentence_tokens
    if current_part:
        parts.append(current_part.strip())
    return parts

# === Prompt-Konfiguration für die verschiedenen Modi ===
if mode == 1:
    messages = [{
        "role": "user",
        "content": f"""Erstelle eine prägnante, übersichtliche Meeting-Zusammenfassung.
- Extrahiere die wichtigsten Punkte und Erkenntnisse.
- Formatiere klar strukturiert im Markdown-Format (klare Überschriften, Absätze, Bulletpoints).
- Nutze die maximal verfügbare Tokenanzahl.

Transkript:
{transcript_content}"""
    }]
elif mode == 2:
    user_prompt = input("Gib deine individuelle Aufgabe ein:\n")
    messages = [{
        "role": "user",
        "content": f"{transcript_content}\n\n{user_prompt}\n\nBitte strukturiere die Antwort übersichtlich im Markdown-Format."
    }]
elif mode == 3:
    schema_prompt = """
Erstelle eine umfassende Meeting-Zusammenfassung mit folgender Struktur:
1. **Grundlegende Metadaten** (Datum, Uhrzeit, Ort, Teilnehmer)
2. **Diskursanalyse** (Sprecher-Dominanz, Emotionen, Intensität der Diskussion)
3. **Entscheidungen & Aufgaben** (mit Begründung, nur belegbare To-Dos, und zusätzliche, optionale Aufgaben als separater Abschnitt)
4. **Follow-up** (Lessons Learned, offene Fragen, Agenda für den nächsten Termin)
Formatiere die Antwort klar strukturiert im Markdown-Format (Überschriften, Absätze, Bulletpoints).
    """
    messages = [{
        "role": "user",
        "content": f"Transkript:\n{transcript_content}\n\n{schema_prompt}"
    }]
elif mode == 4:
    # Punkt 4: Besprechungsprotokoll
    protocol_template = """
Erstelle ein detailiertes Besprechungsprotokoll im Markdown-Format, das folgende Abschnitte beinhaltet:

1. **Kopfzeile**
   - Datum: [TT.MM.JJJJ]
   - Uhrzeit: [HH:MM]
   - Ort: [Besprechungsraum]
   - Teilnehmer: [Liste]

2. **Agenda**
   - Kurze Übersicht der besprochenen Themen
   - Auflistung der einzelnen Punkte

3. **Diskussionspunkte**
   - Detaillierte, strukturierte Zusammenfassung der Diskussion
   - Wichtige Zitate (mit Quellenangabe, z.B. „... Zitat ...“)

4. **Entscheidungen & Aufgaben**
   - Übersicht aller getroffenen Entscheidungen und eindeutig belegbarer To-Dos
   - Separater Abschnitt für *Mögliche von <Modell> erarbeitete Todos* (für weitergehende Ideen)

5. **Offene Fragen / Weiteres**
   - Liste offener Themen für zukünftige Besprechungen

6. **Lessons Learned**
   - Zentrale Erkenntnisse und kritische Einsichten

7. **Stärkste Metaphern / Schlüssel-Aussagen**
   - Die einprägsamsten Formulierungen und Metaphern

8. **Erkenntnisse & Unvollständige Ideen**
   - Zusätzliche Einsichten, die noch nicht abschließend bewertet wurden

9. **Agenda für den nächsten Termin**
   - Vorläufige Punkte basierend auf offenen Fragen

Bitte formatiere alle Abschnitte übersichtlich, mit klaren, fettgedruckten Überschriften und strukturierten Absätzen.

Transkript:
{TRANSCRIPT}
"""
    prompt_template = protocol_template
    if api_choice == "2" and model_choice not in ("4", "8"):
        transcript_parts = split_transcript(transcript_content, filtered_models[model_choice]["max_tokens"], selected_model)
    else:
        transcript_parts = [transcript_content]
    protocol_parts = []
    for i, part in enumerate(transcript_parts):
        current_prompt = prompt_template.replace("{TRANSCRIPT}", part)
        messages = [{"role": "user", "content": current_prompt}]
        print(f"Sende Teil {i+1} von {len(transcript_parts)}...")
        response = client.chat.completions.create(
            model=selected_model if api_choice == "1" else filtered_models[model_choice]["name"],
            messages=messages
        )
        protocol_parts.append(response.choices[0].message.content.strip())
    if len(protocol_parts) > 1:
        protocol = merge_protocol_parts(protocol_parts, client, selected_model)
    else:
        protocol = protocol_parts[0]
elif mode == 5:
    messages = [{
        "role": "user",
        "content": f"""Analysiere das folgende Transkript eines Meetings und erstelle ein übersichtliches Protokoll im Markdown-Format.
Berücksichtige:
- Eindeutige Extraktion der Teilnehmer (Name, Position)
- Strukturierte Darstellung der Inhalte (klare Überschriften, Absätze und Bulletpoints)
- Zusammenfassung der Kernpunkte, Entscheidungen, offenen Fragen, Lessons Learned und Vorschläge für die Agenda des nächsten Treffens

Transkript:
{transcript_content}"""
    }]

# === API-Anfrage und Antwortverarbeitung ===
if api_choice == "1":
    response = client.chat.completions.create(
        model=selected_model,
        messages=messages
    )
    protocol = response.choices[0].message.content.strip()
else:
    if model_choice == "8":
        print("Für benutzerdefinierte Modelle wird das Transkript nicht aufgeteilt.")
        response = client.chat.completions.create(
            model=selected_model,
            messages=messages
        )
        protocol = response.choices[0].message.content.strip()
    else:
        if mode != 4:
            selected_model_data = filtered_models[model_choice]
            max_tokens = selected_model_data["max_tokens"]
            transcript_parts = split_transcript(transcript_content, max_tokens, selected_model_data["name"])
            protocol_parts = []
            for i, part in enumerate(transcript_parts):
                print(f"Sende Teil {i+1} von {len(transcript_parts)}...")
                messages[0]["content"] = messages[0]["content"].replace(transcript_content, part)
                response = client.chat.completions.create(
                    model=selected_model_data["name"],
                    messages=messages
                )
                protocol_parts.append(response.choices[0].message.content.strip())
            protocol = "\n\n".join(protocol_parts)

# === Zusätzlichen Header einfügen: Modell & Prompt-Option ===
prompt_option_names = {
    1: "Standard-Zusammenfassung",
    2: "Individuelle Aufgabe",
    3: "Ultimatives Schema",
    4: "Besprechungsprotokoll",
    5: "Auf eigene Weise"
}
header_line = f"*Erstellt mit Modell: {selected_model} | Prompt: {prompt_option_names[mode]}*"
protocol = header_line + "\n\n" + protocol

# === Dateinamen generieren basierend auf einem vom Modell erarbeiteten Schlagwort ===
name_prompt = f"Fasse das folgende Protokoll in einem prägnanten, einprägsamen Schlagwort zusammen, das als Dateiname verwendet werden kann. Gib nur das Schlagwort zurück:\n\n{protocol}"
name_messages = [{"role": "user", "content": name_prompt}]
name_response = client.chat.completions.create(
    model=selected_model if api_choice == "1" else (selected_model if model_choice == "8" else filtered_models[model_choice]["name"]),
    messages=name_messages
)
filename_keyword = name_response.choices[0].message.content.strip().replace(" ", "-")

datum = datetime.now().strftime("%d.%m")
random_string = ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))
md_filename = f"{filename_keyword}.{datum}#{mode}.{random_string}.md"
html_filename = f"{filename_keyword}.{datum}#{mode}.{random_string}.html"

output_dir = "/content/Transkripte"
os.makedirs(output_dir, exist_ok=True)

def create_html(content):
    html_content = markdown.markdown(content, extensions=['extra'])
    return f"{HTML_STYLES}\n{html_content}"

with open(os.path.join(output_dir, md_filename), "w", encoding="utf-8") as f:
    f.write(protocol)
with open(os.path.join(output_dir, html_filename), "w", encoding="utf-8") as f:
    f.write(create_html(protocol))

print(f"\nErgebnisse gespeichert als:")
print(f"- {os.path.join(output_dir, md_filename)}")
print(f"- {os.path.join(output_dir, html_filename)}")
files.download(os.path.join(output_dir, md_filename))
files.download(os.path.join(output_dir, html_filename))